In [1]:
from pymatgen import Structure, MPRester, Lattice
# from math import sin, cos, sqrt, pi
from pymatgen.core.surface import SlabGenerator, Slab
from pymatgen.vis.structure_vtk import StructureVis
from atomate.vasp.fireworks.core import OptimizeFW
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer as sga
mpr = MPRester('xGTZsWPN3BoydaL4')
from pymatgen.io.vasp.inputs import Poscar

/Users/jordanburns/miniconda3/envs/dev/lib/python3.7/site-packages/pymatgen/io/cif.py:44: UserWarning: Please install optional dependency pybtex if youwant to extract references from CIF files.
  warnings.warn("Please install optional dependency pybtex if you"


In [2]:
# import Li2MnO3 structure from materials project 
struct = mpr.get_structure_by_material_id("mp-18988", conventional_unit_cell=True)

In [3]:
sga = sga(struct)

In [4]:
struct_sym = sga.get_symmetrized_structure()

In [5]:
wyckoff_symbols = struct_sym.wyckoff_symbols
print(wyckoff_symbols)

['4h', '2b', '2c', '4g', '8j', '4i']


In [6]:
equiv_indices = struct_sym.equivalent_indices
wyckoff_labels = ['']*len(struct_sym)
for wyckoff_symbol, equiv_group in zip(wyckoff_symbols, equiv_indices):
    for idx in equiv_group:
        wyckoff_labels[idx] = wyckoff_symbol
print(wyckoff_labels)

['4h', '4h', '2b', '2c', '4h', '4h', '2b', '2c', '4g', '4g', '4g', '4g', '8j', '8j', '4i', '4i', '8j', '8j', '8j', '8j', '4i', '4i', '8j', '8j']


In [8]:
struct_sym.add_site_property('wyckoff', wyckoff_labels)
struct_sym = Structure.from_sites(struct_sym.sites)

In [9]:
slabgen_wyckoff = SlabGenerator(struct_sym, [1,1,0], 10 , 10, in_unit_planes=True)
slabs_wyckoff = slabgen_wyckoff.get_slabs()
all_slabs_corrected_wyckoff = [slabgen_wyckoff.nonstoichiometric_symmetrized_slab(slab)
                               for slab in slabs_wyckoff]
all_slabs_corrected_wyckoff = sum(all_slabs_corrected_wyckoff, [])

In [10]:
from pymatgen.analysis.bond_valence import BVAnalyzer
bva_analyzer = BVAnalyzer()

In [11]:
bva_struct = bva_analyzer.get_oxi_state_decorated_structure(struct_sym)

In [12]:
slabgen_bva_before_sym = SlabGenerator(bva_struct, [1,1,0], 10 , 10, in_unit_planes=True)
slabs_bva_before = slabgen_bva_before_sym.get_slabs()

In [14]:
slabgen_bva = SlabGenerator(bva_struct, [1,1,0], 10 , 10, in_unit_planes=True)
slabs_bva = slabgen_bva.get_slabs()
all_slabs_corrected_bva = [slabgen_wyckoff.nonstoichiometric_symmetrized_slab(slab)
                               for slab in slabs_bva]
all_slabs_corrected_bva = sum(all_slabs_corrected_bva, [])

In [21]:
move_struct = Structure.from_dict(slabs_bva_before[0].as_dict())

In [23]:
move_struct_copy = move_struct.copy()

In [94]:
surface_sites = slabs_bva_before[0].get_surface_sites()

In [22]:
slabgen_move = SlabGenerator(move_struct, [1,1,0], 10, 10, in_unit_planes = True)

In [95]:
moved = slabgen_move.move_to_other_side(slabs_bva_before[0],[surface_sites['top'][0][1]])

In [91]:
new_surface_sites = moved.get_surface_sites()

In [92]:
new_surface_sites  

{'top': [[PeriodicSite: Li+ (0.4911, 1.7227, 80.3645) [0.0399, 0.3431, 0.9686],
   0],
  [PeriodicSite: Li+ (4.7042, 4.4353, 81.0878) [0.7901, 0.8834, 0.9767], 2],
  [PeriodicSite: Li+ (3.0472, 1.2973, 78.9850) [0.5651, 0.2584, 0.9517], 3],
  [PeriodicSite: Li+ (0.5997, 0.8720, 77.5516) [0.0904, 0.1737, 0.9347], 5],
  [PeriodicSite: Mn4+ (2.2116, 4.0175, 79.6787) [0.3061, 0.8002, 0.9600], 40],
  [PeriodicSite: Mn4+ (2.1931, 4.8531, 82.4429) [0.2741, 0.9666, 0.9933], 41],
  [PeriodicSite: Mn4+ (4.7326, 3.5980, 78.3181) [0.8241, 0.7166, 0.9433], 43],
  [PeriodicSite: O2- (5.6159, 5.0163, 79.3043) [0.9527, 0.9991, 0.9550], 61],
  [PeriodicSite: O2- (3.8185, 2.9397, 79.9078) [0.6637, 0.5855, 0.9626], 62],
  [PeriodicSite: O2- (4.7400, 0.9102, 82.2410) [0.9165, 0.1813, 0.9907], 63],
  [PeriodicSite: O2- (3.7925, 3.8543, 82.8712) [0.6276, 0.7677, 0.9983], 64],
  [PeriodicSite: O2- (2.0706, 0.4441, 80.7322) [0.3988, 0.0885, 0.9729], 65],
  [PeriodicSite: O2- (4.0237, 2.1506, 77.2378) [0.7314,

In [90]:
moved = slabgen_move.move_to_other_side(moved,[70])
moved.dipole

array([-0.67043431, -0.21753195, 62.2091293 ])

In [74]:
Poscar(moved)

Li40 Mn20 O60
1.0
5.003584 0.000000 0.053924
0.849877 5.020732 0.026716
0.000000 0.000000 82.956154
Li Mn O
40 20 60
direct
0.039866 0.343113 0.468623 Li+
0.540360 0.423679 0.484736 Li+
0.790113 0.883396 0.476679 Li+
0.565113 0.258396 0.451679 Li+
0.589866 0.093113 0.918623 Li+
0.090360 0.173679 0.934736 Li+
0.340113 0.633396 0.926679 Li+
0.115113 0.008396 0.901679 Li+
0.139866 0.843113 0.868623 Li+
0.640360 0.923679 0.884736 Li+
0.890113 0.383396 0.876679 Li+
0.665113 0.758396 0.851679 Li+
0.689866 0.593113 0.818623 Li+
0.190360 0.673679 0.834736 Li+
0.440113 0.133396 0.826679 Li+
0.215113 0.508396 0.801679 Li+
0.239866 0.343113 0.768623 Li+
0.740360 0.423679 0.784736 Li+
0.990113 0.883396 0.776679 Li+
0.765113 0.258396 0.751679 Li+
0.789866 0.093113 0.718623 Li+
0.290360 0.173679 0.734736 Li+
0.540113 0.633396 0.726679 Li+
0.315113 0.008396 0.701679 Li+
0.339866 0.843113 0.668623 Li+
0.840360 0.923679 0.684736 Li+
0.090113 0.383396 0.676679 Li+
0.865113 0.758396 0.651679 Li+
0.889866

In [93]:
slabs_bva_before[0].dipole

array([  0.2234781 ,   0.07251065, -20.73637643])

In [101]:
moved64 = slabgen_move.move_to_other_side(slabs_bva_before[0],[64])

In [102]:
moved64.dipole

array([-0.67043431, -0.21753195, 62.2091293 ])

In [103]:
surfacess = moved64.get_surface_sites()

In [104]:
surfacess

{'top': [[PeriodicSite: Li+ (3.0638, 2.1272, 81.7303) [0.5404, 0.4237, 0.9847],
   1],
  [PeriodicSite: Li+ (4.7042, 4.4353, 81.0878) [0.7901, 0.8834, 0.9767], 2],
  [PeriodicSite: Mn4+ (2.1931, 4.8531, 82.4429) [0.2741, 0.9666, 0.9933], 41],
  [PeriodicSite: O2- (1.4842, 3.4058, 81.3626) [0.1814, 0.6783, 0.9805], 60],
  [PeriodicSite: O2- (4.7400, 0.9102, 82.2410) [0.9165, 0.1813, 0.9907], 63],
  [PeriodicSite: O2- (2.0706, 0.4441, 80.7322) [0.3988, 0.0885, 0.9729], 65]],
 'bottom': [[PeriodicSite: Li+ (5.0320, 0.4675, 43.0788) [0.9899, 0.0931, 0.5186],
   36],
  [PeriodicSite: Li+ (2.5845, 0.0422, 41.6454) [0.5151, 0.0084, 0.5017], 39],
  [PeriodicSite: Mn4+ (1.7490, 2.7623, 42.3390) [0.2561, 0.5502, 0.5100], 58],
  [PeriodicSite: O2- (3.7925, 3.8543, 41.3931) [0.6276, 0.7677, 0.4983], 64],
  [PeriodicSite: O2- (1.0216, 2.1506, 44.0230) [0.1314, 0.4283, 0.5305], 114],
  [PeriodicSite: O2- (5.1532, 3.7611, 41.9647) [0.9027, 0.7491, 0.5050], 115],
  [PeriodicSite: O2- (3.3559, 1.6845, 

In [105]:
surfacess_before = slabs_bva_before[0].get_surface_sites()

In [106]:
surfacess_before

{'top': [[PeriodicSite: Li+ (3.0638, 2.1272, 81.7303) [0.5404, 0.4237, 0.9847],
   1],
  [PeriodicSite: Mn4+ (2.1931, 4.8531, 82.4429) [0.2741, 0.9666, 0.9933], 41],
  [PeriodicSite: O2- (1.4842, 3.4058, 81.3626) [0.1814, 0.6783, 0.9805], 60],
  [PeriodicSite: O2- (4.7400, 0.9102, 82.2410) [0.9165, 0.1813, 0.9907], 63],
  [PeriodicSite: O2- (3.7925, 3.8543, 82.8712) [0.6276, 0.7677, 0.9983], 64],
  [PeriodicSite: O2- (2.0706, 0.4441, 80.7322) [0.3988, 0.0885, 0.9729], 65]],
 'bottom': [[PeriodicSite: Li+ (5.0320, 0.4675, 43.0788) [0.9899, 0.0931, 0.5186],
   36],
  [PeriodicSite: Li+ (2.5845, 0.0422, 41.6454) [0.5151, 0.0084, 0.5017], 39],
  [PeriodicSite: Mn4+ (1.7490, 2.7623, 42.3390) [0.2561, 0.5502, 0.5100], 58],
  [PeriodicSite: O2- (1.0216, 2.1506, 44.0230) [0.1314, 0.4283, 0.5305], 114],
  [PeriodicSite: O2- (5.1532, 3.7611, 41.9647) [0.9027, 0.7491, 0.5050], 115],
  [PeriodicSite: O2- (3.3559, 1.6845, 42.5682) [0.6137, 0.3355, 0.5126], 116],
  [PeriodicSite: O2- (2.4579, 4.2097

In [113]:
moved1 = slabgen_move.move_to_other_side(slabs_bva_before[0],[1])

In [114]:
moved1.dipole

array([  0.67043431,   0.21753195, -62.2091293 ])

In [116]:
moved1.get_surface_sites()

{'top': [[PeriodicSite: Li+ (0.4911, 1.7227, 80.3645) [0.0399, 0.3431, 0.9686],
   0],
  [PeriodicSite: Li+ (4.7042, 4.4353, 81.0878) [0.7901, 0.8834, 0.9767], 2],
  [PeriodicSite: Li+ (3.0472, 1.2973, 78.9850) [0.5651, 0.2584, 0.9517], 3],
  [PeriodicSite: Mn4+ (2.1931, 4.8531, 82.4429) [0.2741, 0.9666, 0.9933], 41],
  [PeriodicSite: O2- (1.4842, 3.4058, 81.3626) [0.1814, 0.6783, 0.9805], 60],
  [PeriodicSite: O2- (3.8185, 2.9397, 79.9078) [0.6637, 0.5855, 0.9626], 62],
  [PeriodicSite: O2- (4.7400, 0.9102, 82.2410) [0.9165, 0.1813, 0.9907], 63],
  [PeriodicSite: O2- (3.7925, 3.8543, 82.8712) [0.6276, 0.7677, 0.9983], 64],
  [PeriodicSite: O2- (2.0706, 0.4441, 80.7322) [0.3988, 0.0885, 0.9729], 65]],
 'bottom': [[PeriodicSite: Li+ (3.0638, 2.1272, 40.2523) [0.5404, 0.4237, 0.4847],
   1],
  [PeriodicSite: Li+ (5.0320, 0.4675, 43.0788) [0.9899, 0.0931, 0.5186], 36],
  [PeriodicSite: Li+ (2.5845, 0.0422, 41.6454) [0.5151, 0.0084, 0.5017], 39],
  [PeriodicSite: Mn4+ (1.7490, 2.7623, 42.3